In [1]:
import sys
sys.path.append("../../../..")
%load_ext autoreload
%autoreload 1
import lightning.pytorch as pl
import numpy as np

from ntupleReaders.clue_ntuple_reader import ClueNtupleReader

from ml.dynamic_reduction_network import DynamicReductionNetwork
%aimport ml.regression.drn.modules
%aimport ml.regression.drn.dataset_making
%aimport energy_resolution.sigma_over_e
from ml.regression.drn.modules import *
from ml.regression.drn.dataset_making import *
%aimport ml.regression.drn.callbacks.sigma_over_e
from ml.regression.drn.callbacks.sigma_over_e import *
%aimport ml.regression.drn.callbacks.simple_plots
from ml.regression.drn.callbacks.simple_plots import *

In [2]:
reader = ClueNtupleReader("v41", "cmssw", "data") # sim_proton_v46_patchMIP
datamodule = DRNDataModule(reader, datasetComputationClass=LayerClustersTensorMaker, multiprocess_loader=False)

class MockDRN(nn.Module):
    def __init__(self, *args, **kwargs) -> None:
        super().__init__(*args, **kwargs)
        self.param = nn.Parameter(torch.tensor([1.]))
    def forward(self, x):
        return torch.normal(mean=torch.ones(x.num_graphs, device=x.x.device), std=torch.ones(x.num_graphs, device=x.x.device)*0.05)  * self.param

model = DRNModule(drn=MockDRN(), loss=RatioCorrectedLoss([-0.2597882 , -0.24326517,  1.01537901], simulation=False), optimizer=torch.optim.AdamW)
devices = [3]

In [9]:
callbacks = [
    SigmaOverECallback( overlaySigmaOverEResults=[reader.loadSigmaOverEResults("rechits"), reader.loadSigmaOverEResults("clue3d")], 
    every_n_epochs=1, fit_data="full", debug_mode=False, train_datatype="data"),

    SimplePlotsCallback(simulation=False)
]
#callback.on_validation_epoch_end(trainer, model)

In [10]:
trainer = pl.Trainer(accelerator="gpu", devices=devices, enable_progress_bar=True, fast_dev_run=False, logger=True, detect_anomaly=False, 
                     callbacks=callbacks, #limit_train_batches=0.09, limit_val_batches=0.25,
                     max_epochs=1)

trainer.fit(model=model, datamodule=datamodule)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]

  | Name | Type    | Params
---------------------------------
0 | drn  | MockDRN | 1     
---------------------------------
1         Trainable params
0         Non-trainable params
1         Total params
0.000     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

/grid_mnt/vol_home/llr/cms/cuisset/mambaforge/envs/clustering-analysis-gpu2/lib/python3.10/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:430: PossibleUserWarning: The dataloader, val_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 96 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/grid_mnt/vol_home/llr/cms/cuisset/mambaforge/envs/clustering-analysis-gpu2/lib/python3.10/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:430: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 96 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/grid_mnt/vol_home/llr/cms/cuisset/mambaforge/envs/clustering-analysis-gpu2/lib/python3.10/s

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Start predicting!
Start fitting!


`Trainer.fit` stopped: `max_epochs=1` reached.


In [5]:
plotFittedMean(callbacks[0].overlaySigmaOverEResults + [callbacks[0].sigmaOverEPlotter.plotElt], errors=True, beamEnergiesToCircle=True)

<Figure size 1000x1000 with 1 Axes>

In [6]:
datamodule.prepare_data()
datamodule.setup("predict")
beamEnergies = []
tracksterEnergy_bare = []
tracksterEnergy_pred = []
count = 0
for batch in datamodule.train_dataloader():
    beamEnergies.append(batch.beamEnergy)
    tracksterEnergy_bare.append(batch.tracksterEnergy)
    tracksterEnergy_pred.append(batch.tracksterEnergy*torch.normal(torch.ones_like(batch.tracksterEnergy), batch.beamEnergy*0.0001))
    count += len(batch.beamEnergy)
    if count > 1e4:
        break

In [7]:
beamEnergies = convertListOfTensorsToNumpy(beamEnergies)
tracksterEnergy_bare = convertListOfTensorsToNumpy(tracksterEnergy_bare)
tracksterEnergy_pred = convertListOfTensorsToNumpy(tracksterEnergy_pred)

In [8]:
doubleViolin(beamEnergies, tracksterEnergy_bare, tracksterEnergy_pred,overlapViolins=False, normBy="incidentBeamEnergy")

(<Figure size 1000x500 with 1 Axes>,
 <Axes: xlabel='Beam energy (GeV)', ylabel='Ratio of (predicted) trackster energy over incident beam energy'>)